In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import pickle
#from numba import cuda
from scipy import spatial, sparse
import random as rd
from collections import Counter

In [34]:
data = pd.read_csv('~/fires_merged_weather.csv', index_col=0,
                  #dtype for smaller representation
                  dtype={'STAT_CAUSE_DESCR': 'category', 'STATE': 'category', 'DISCOVERY_MONTH': 'category',
                        'Fog': 'bool', 'FunnelCloud': 'bool', 'Hail': 'bool', 'Rain': 'bool',
                        'Snow': 'bool', 'Thunder': 'bool'}
                  )

In [46]:
data = data[data['FIRE_YEAR']==2015]

In [48]:
data.reset_index(drop=True,inplace=True)

In [49]:
data['DAY'] = (data['FIRE_YEAR']-1992)*365+data['DISCOVERY_DOY']

In [50]:
data = data.sort_values('DAY', ascending=True, kind='mergesort')

In [26]:
order = dict(zip(range(data.shape[0]),data.index))

In [27]:
attr_time = data[["DAY"]]
attr_space = data[["LATITUDE", "LONGITUDE"]]

In [28]:
%%time
Aboth_15 = get_A_both(attr_space, 1.0, attr_time, 1.01, blocksize=100,ordering=order)

Setting up...
Initializing sparse matrix...
CPU times: user 5.86 s, sys: 51.8 ms, total: 5.91 s
Wall time: 5.92 s


In [18]:
import os
os.chdir('/home/jime/wildfire_GNN/adj')

In [29]:
with open('A_both_15_unsorted.pkl', 'rb') as f:
    pickle.dump(Aboth_15, f)

In [30]:
with open('A_both_unordered.pkl', 'rb') as f:
    old_Aboth = pickle.load(f)

In [32]:
Aboth_15-old_Aboth

<74491x74491 sparse matrix of type '<class 'numpy.float64'>'
	with 3543722 stored elements in Compressed Sparse Row format>

In [39]:
Aboth_sorted[1].todense()

matrix([[0., 1., 0., ..., 0., 0., 0.]])

In [40]:
Aboth[12017].todense()

matrix([[0., 1., 0., ..., 0., 0., 0.]])

In [12]:
def get_block(values, dist, row_start, row_end, col_start, col_end):
    
    N = values.shape[0]

    # Get the relevant values for this block
    #print("Get row/col vals")
    row_values = values[row_start:row_end, :]
    col_values = values[col_start:col_end, :]

    # Get distance matrix for this block
    #print("Calculate distance")
    # use cuda implementation from stackoverflow
    D = spatial.distance.cdist(row_values, col_values)

    # Threshold it
    #print("Threshold it")
    subA = D <= dist
    
    return subA

def get_A_both(data_space, dist_space, data_time, dist_time, blocksize=10000, ordering=None):
    
    print("Setting up...")
    space_values = np.array(data_space.values)
    time_values = np.array(data_time.values)
    
    # Dimensions should be be (N, K), even if K = 1 columns. Reshape if needed
    if len(space_values.shape) == 1:
        space_values = space_values.reshape((space_values.shape[0], 1))
    if len(time_values.shape) == 1:
        time_values = time_values.reshape((time_values.shape[0], 1))
        
    assert space_values.shape[0] == time_values.shape[0], "Datasets must have same number of observations"
    N = space_values.shape[0]
        
    print("Initializing sparse matrix...")
    # Initialize sparse matrix
    A = sparse.lil_matrix((N,N))
    
    # Divide-and-conquer: split the overall big adjacency matrix into
    # blocksize x blocksize chunks, then use scipy's super-fast C implementation for distance matrix
    for i in range(N // blocksize+1):
        skip_row = False
            
        for j in range(i, N // blocksize+1):
            
            if skip_row:
                continue
            
            # Make sure we don't go out of bounds if N isn't divisible by blocksize!
            row_start = i * blocksize
            row_end   = min((i+1) * blocksize, N-1)
            col_start = j * blocksize
            col_end   = min((j+1) * blocksize, N-1)
            
            sub_A_time = get_block(time_values, dist_time, row_start, row_end, col_start, col_end)
            
            if (sub_A_time == 0).all():
                #print((i,j))
                #print("No more non-zeroes column-wise on this row!  Skipping to next")
                skip_row = True
                
            sub_A_space = get_block(space_values, dist_space, row_start, row_end, col_start, col_end)
            
            # Insert into matrix
            #print("Insert into matrix")
            subA = sub_A_time * sub_A_space            
            
            A[row_start:row_end, col_start:col_end] = subA
            
            # This graph is undirected--A will be symmetric! So set the other side now
            #if i != j:
                #print("Insert into matrix, transposed")
                #A[col_start:col_end, row_start:row_end] = subA.T

    # Convert to CSR format for fast arithmetic
    if ordering!=None:
        row = np.array(sparse.find(A)[0])
        row = [ordering[i] for i in row]
        col = np.array(sparse.find(A)[1])
        entries = np.array(sparse.find(A)[2])
        A = sparse.csr_matrix( (entries, (row,col)), shape=(A.shape[0],A.shape[1]) )
    else:
        A = A.tocsr()
    
    # This graph is undirected--so set the other side to be symmetric
    # Do this efficiently by adding the transpose, and then removing the elements equal to 2 to 1
    A += A.T
    A[A == 2] = 1
            
    return A